In [4]:
# TODO: llm修改毛病 （主要是格式，便于 正则匹配）
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

llm = ChatOpenAI(
    api_key="sk-EQY6oy2D9Brb2IWxymHTnmeFz5unIyobzHtjDgFs2ZwEPjmY",
    base_url="https://api.chatanywhere.tech/v1",
    model="gpt-4o-mini",
    temperature=0.35,
)
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate
)

In [5]:
import re
import tiktoken
def split_text(text, max_tokens=4000, model="gpt-3.5-turbo"):
    """
    将文本拆分成不超过 max_tokens 的块，确保不拆分句子。
    """
    enc = tiktoken.encoding_for_model(model)
    sentences = re.split(r'(。|!|\?|\n)', text)  # 按句号、感叹号、问号分割
    sentences = [s.strip() for s in sentences if s.strip()]  # 清理空白字符
    
    chunks = []
    current_chunk = []
    current_length = 0
    
    for sentence in sentences:
        sentence_tokens = len(enc.encode(sentence))
        if current_length + sentence_tokens > max_tokens:
            if current_chunk:
                chunks.append(" ".join(current_chunk))
                current_chunk = []
                current_length = 0
        
        if sentence_tokens > max_tokens:
            # 处理单个句子超过 max_tokens 的情况
            words = sentence.split()
            sub_chunk = []
            sub_length = 0
            for word in words:
                word_tokens = len(enc.encode(word))
                if sub_length + word_tokens > max_tokens:
                    chunks.append(" ".join(sub_chunk))
                    sub_chunk = []
                    sub_length = 0
                sub_chunk.append(word)
                sub_length += word_tokens
            if sub_chunk:
                chunks.append(" ".join(sub_chunk))
        else:
            current_chunk.append(sentence)
            current_length += sentence_tokens
    
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    
    return chunks

In [6]:
def split_by_title(text):
    pattern = r"(?:(?<=\n)|(?<=\s)|(?<=^))([一二三四五六七八九十])、"
    parts = re.split(pattern, text)
    chunks = [parts[0]]
    for i in range(1, len(parts), 2):
        tmp = parts[i] + '、' + parts[i + 1]
        chunks.append(tmp)    
    return chunks

In [7]:
system_template="""
-目标- 
给定文本文档，更正文档中可能存在的识别错误，剔除文档中的例题。

-步骤-
1.文档中可能存在的错误：文档中可能存在的错误包括缺字漏字、错别字。
2.文档中可能存在的习题与解析，需要识别并去除习题：文档中存在的 即学即练、走近真题、【名家精析】、【参考答案】、(单选)、(多选)、(名词解释)等部分（整体的模块，而非单独的标记），这些部分是需要被去除的。
4.文档中可能存在以latex公式形式出现的标号或文本格式，如$\textcircled{{1}}$、$\\bigcirc$、需要更正为正常文本形式。其中$\textcircled{{1}}$更正为①，\sim更正为~，其他latex格式文本也用相应的形式纠正。
这些文本需要纠正，不能直接丢弃。
5.文档中可能存在中英文标点符号不匹配的情况，需要结合上下文统一标点符号。
6.文档中可能存在的数字、日期、单位等需要更正为正确的形式。

-注意事项-
1.文本标题后存在的'【单选】'、'【填空】'等标记需要保留。
2.文本小标题格式有 一、二、 (一)(二) 1. 2. (1) (2) 等，不要改变小标题格式，尤其是前面的编号，并且每个小标题后应该换行。
3.'辩一辩'、'学一学'等标记需要去除。
4.不要改变文档的原有结构，只需更正文档中可能存在的识别错误，剔除文档中的例题。
5.除了以上提到的情况，不要更改文档中的内容。

-示例-
######################
示例1：
---------输入----------
第一节 教育制度概述 一、制度的概念【单选】关于制度的内涵中外学者各抒己见，有多种说法。
走近真题 (单选)孔子的教育思想集中体现在 ( B.《中庸》C.《论语》D.《孟子》 【精析】C孔子的教育思想集中体现在《论语》中。
二、教育制度的定义与构成要素【单选、填空】教育制度的定义与构成要素包括教育目的、教育方针、教育内容、教育方法、教育手段、教育组织、教育管理等。
即学即练 (单选)主张教育的一切活动都应服务于社会需要，教育除了社会目的以外并无其他目的的观点属于
A.个人本位论 B.社会本位论 C.文化本位论 D.生活本位论
2010年，党中央、国务院正式颁布《国家中长期教育改革和发展规划纲要(2010一2020年)》简称《教育规划纲要》)
我的笔记
4.未来我国教育目的应有的定位一—培养“人”
【名家精析】B 题干所述为社会本位论的观点。
---------输出----------
第一节 教育制度概述
一、制度的概念 【单选】
关于制度的内涵中外学者各抒己见，有多种说法。
二、教育制度的定义与构成要素 【单选、填空】
教育制度的定义与构成要素包括教育目的、教育方针、教育内容、教育方法、教育手段、教育组织、教育管理等。
2010年，党中央、国务院正式颁布《国家中长期教育改革和发展规划纲要(2010-2020年)》(简称《教育规划纲要》)
4.未来我国教育目的应有的定位————培养“人”
######################
示例2：
---------输入----------
(单选)英国政府1870年颁布的《初等教育法》属于
A.双轨型学制 B.单轨型学制C.中间型学制 D.分支型学制【参考答案】A
$\circledcirc$ 论教育对象 “有教无类”
孔子创办私学，主张“有教无类”，即不分贵族与平民，都可以接受教育，成为教育的对象，打破了贵族垄断教育的局面。
$\textcircled{{3}}$ 论培养目标——“学而优则仕”
孔子的教育目的就是要培养德才兼备的君子。“学而优则仕”口号的提出，确定了培养统治人才这一教育目的，在教育史上有重要的意义。
$\circledast$ 论教育内容—- “仁”“礼”为核心
孔子以“礼”为道德规范，以“仁”为最高道德标准，主张立志、克己、内省、力行、改过，开创了以儒学伦理为核心，以道德教育为本的教育传统。孔子整理“六经”，即《诗》书》礼》《乐》易》春秋》六种教材，奠定了儒家教育内容的基础。在
教育内容上，孔子主张“子以四教:文、行、忠、信”。孔子的教学内容偏重社会人事、偏重文事，轻视科技与生产劳动。
我的笔记 $\textcircled{{5}}$ 教学纲领——“博学于文，约之以礼”
$\circled{{6}}$ 论教育方法 第一，启发诱导。孔子曾说过：“不愤不启，不不发。举一隅不以三隅反，则不复也。"孔子是世界上最早提出启发式教学的教育家。
第二，因材施教。孔子因材施教的前提是承认学生间的个体差异，并了解学生特点，在了解的基础上有针对性地进行教育。如“求也退，故进之；由也兼人，故退之。”
$\textcircled{{7}}$ 论学习方法
---------输出----------
②论教育对象——“有教无类”
孔子创办私学，主张“有教无类”，即不分贵族与平民，都可以接受教育，成为教育的对象，打破了贵族垄断教育的局面。
③论培养目标——“学而优则仕”
孔子的教育目的就是要培养德才兼备的君子。“学而优则仕”口号的提出，确定了培养统治人才这一教育目的，在教育史上有重要的意义。
④论教育内容——“仁”“礼”为核心
孔子以“礼”为道德规范，以“仁”为最高道德标准，主张立志、克己、内省、力行、改过，开创了以儒学伦理为核心，以道德教育为本的教育传统。孔子整理“六经”，即《诗》书》礼》《乐》易》春秋》六种教材，奠定了儒家教育内容的基础。在
教育内容上，孔子主张“子以四教:文、行、忠、信”。孔子的教学内容偏重社会人事、偏重文事，轻视科技与生产劳动。
⑤教学纲领——“博学于文，约之以礼”
⑥论教育方法
第一，启发诱导。孔子曾说过：“不愤不启，不不发。举一隅不以三隅反，则不复也。”孔子是世界上最早提出启发式教学的教育家。
第二，因材施教。孔子因材施教的前提是承认学生间的个体差异，并了解学生特点，在了解的基础上有针对性地进行教育。如“求也退，故进之；由也兼人，故退之。”
⑦论学习方法
######################
示例3：
---------输入----------
第二节 中国教育制度 小库提醒 小库导学 旧中国的学制沿革 我的笔记 小库课堂 走近真题 1.(单选)我国历史上第一个颁布但没有实施的现代学制是( )A.癸卯学制B.壬寅学制C.壬戌学制D.壬子癸丑学制
【精析】B题干所述为“壬寅学制”。 2.(填空)从世界范围说，现代教育制度始于十八、十九世纪欧美国家的
【参考答案】义务教育 
—————————输出—————————
第二节 中国教育制度
######################
-输入-

"""
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

human_template="""
-真实数据- 
######################
文本：{input_text} 
###################### 
输出：
"""

from langchain.chains import LLMChain
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_template])

chain = LLMChain(llm=llm, prompt=chat_prompt)
# with open(r"D:\code\agent_demo\kg\output\11_第二节 中国教育制度.txt", "r") as f:
#     input = f.read()
#     input = input.replace('{', '{{').replace('}', '}}')
    
#     # txt = split_text(input)
#     txt = split_by_title(input)
#     ans = ""
#     for text in txt:
#         answer = chain.invoke({
#             "input_text": text
#             })
#         print(answer['text'])
#         ans += answer['text']
#     # 保存
#     with open(r"D:\code\agent_demo\kg\clear_out\11_第二节 中国教育制度.txt", "w") as f:
#         f.write(ans)

C:\Users\pixiu\AppData\Local\Temp\ipykernel_10872\573330403.py:95: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=chat_prompt)


In [8]:
file_floder = r"D:\code\agent_demo\kg\output"
def get_files(path):
    import os
    files = os.listdir(path)
    return [os.path.join(path, file) for file in files if file.endswith('.txt')]
files = get_files(file_floder)

for file in files:
    with open(file, "r") as f:
        input = f.read()
        input = input.replace('{', '{{').replace('}', '}}')
        
        txt = split_by_title(input)
        ans = ""
        for text in txt:
            answer = chain.invoke({
                "input_text": text
                })
            # print(answer['text'])
            ans += '\n' + answer['text']
        # 保存
        with open(file.replace('output', 'clear_out_v3'), "w") as f:
            f.write(ans)

In [9]:
filep2 = r"D:\code\agent_demo\kg\clear_out_v3"
files2 = get_files(filep2)
for file in files2:
    with open(file, "r") as f:
        txt = f.read()
        txt = txt.replace('  ', ' ').replace(' \n', '\n').replace('\n\n', '\n').replace('. ', '.')
    
    with open(file, "w") as f:
        f.write(txt)
print('done')

done


In [10]:
filep2 = r"D:\code\agent_demo\kg\clear_out_v2"
files2 = get_files(filep2)
txt_content = ""
for file in files2:
    with open(file, "r") as f:
        txt = f.read()
        txt = txt.replace('\n\n', '\n').replace('. ', '.')
    
    with open(file, "w") as f:
        f.write(txt)
print('done')

done
